In [12]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
%run trips_duration.ipynb

object


/var/folders/mv/1xmk4xds7d548s_3ktkk13600000gn/T/ipykernel_44776/1627526132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
def merging_data(final_1plus_df):

    tourist_num = final_1plus_df
    tourist_num_copy = final_1plus_df.copy()

    min_trips = tourist_num['NumTrips'].min()
    max_trips = tourist_num['NumTrips'].max()

    tourist_num['Tourism St.'] = (tourist_num['NumTrips'] - min_trips) / (max_trips - min_trips)

    tourist_num = pd.DataFrame(final_1plus_df, columns = ['Country', 'Year', 'Tourism St.'])
    tourist_num.sort_values(by = ["Year"])

    unfiltered_predicted_tourist =pd.DataFrame(tourist_num_copy, columns = ["Country", "Year", "NumTrips"])

    return tourist_num, unfiltered_predicted_tourist


In [4]:
datas = merging_data(final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data_not_st

,Country,Year,NumTrips
0,Albania,2017,8186685
1,Albania,2019,10176614
2,Albania,2020,6624361
3,Austria,2012,36003256
4,Austria,2013,36479782
...,...,...,...
345,Switzerland,2018,40810491
346,Switzerland,2019,36986339
347,Switzerland,2020,31748407
348,Switzerland,2021,32989206


In [16]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['Bias'] = 1
    min_year = country_df['Year'].min()
    
    index_country = country_df.index[country_df['Year'].astype(int) == min_year + 3]
    index = index_country[0]

    y_vector = []
    x_matrix = []
    x_list = []
    for i in range(index, len(country_df)): 
        if i - lag < 0:
            continue
        bias = 1 
        country = country_df.iloc[i]["Country"]
        lag_values = [country_df.iloc[i - j]['Tourism St.'] for j in range(1, lag + 1)]
        covid = country_df.iloc[i]['Covid']
        year = country_df.iloc[i]['Year']
        
        x_list = [bias, country, year, covid] + lag_values
        x_matrix.append(x_list)
        y_vector.append(country_df.iloc[i]["Tourism St."])

    x_matrix = pd.DataFrame(x_matrix)
    x_matrix.rename(columns={0: "Bias", 1: "Country", 2:"Year", 3:"Covid", 4:"lag1", 5:"lag2", 6:"lag3"}, inplace=True)


    return x_matrix, y_vector

In [17]:
data[data['Country']=='Albania']

,Country,Year,Tourism St.
0,Albania,2017,0.014321
1,Albania,2019,0.018106
2,Albania,2020,0.011349


In [18]:
X, y = get_lag_columns(data[data['Country'] == 'Albania'], lag=3)
print(X)
print(y)

Empty DataFrame
Columns: []
Index: []
[]


In [19]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.inv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [20]:
def predict_next_year(beta, X):
    return np.dot(beta, X)

In [21]:
def get_country_years_pred(data, country, lag = 3):

    country_df = data[data['Country'] == country]
    country_df, y = get_lag_columns(country_df, lag = lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias', 'Covid'] + lag_cols
    
    X = country_df[feature_cols].values

    beta_tourism = fit_regression(X, y)

    predictions = []
    for index, row in country_df.iterrows():
        year = row['Year']
        X_year = np.array([row[col] for col in feature_cols])
        pred_tourism = predict_next_year(beta_tourism, X_year)

        predictions.append({
            'Country': country,
            'Year': year,
            'Actual_Tourism': y[index],
            'Predicted_Tourism': pred_tourism

        })

    all_years_results = pd.DataFrame(predictions)

    return all_years_results

In [22]:
prev_year_result = get_country_years_pred(data, "Slovakia")
prev_year_result

,Country,Year,Actual_Tourism,Predicted_Tourism
0,Slovakia,2015,0.025319,0.029939
1,Slovakia,2016,0.030385,0.031425
2,Slovakia,2017,0.034212,0.035546
3,Slovakia,2018,0.039610,0.041058
4,Slovakia,2019,0.044606,0.047150
5,Slovakia,2020,0.019206,0.024017
6,Slovakia,2021,0.023941,0.019130
7,Slovakia,2022,0.041598,0.040566
8,Slovakia,2023,0.048077,0.038123


In [23]:
r2_score(prev_year_result['Actual_Tourism'], prev_year_result['Predicted_Tourism'])

0.7782708312971979

In [24]:
def get_country_prediction(data, country, lag= 3):
    country_df = data[data['Country'] == country].copy()
    country_df, y = get_lag_columns(country_df, lag=lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias', 'Covid'] + lag_cols

    X = country_df[feature_cols].values

    beta_tourism = fit_regression(X, y)
    #print(country, beta_tourism)
    last_year = country_df['Year'].max()
    #print(last_year)

    last_row = country_df[country_df['Year'] == last_year].iloc[0]
    lag_values = [last_row[f'lag{i}'] for i in range(1, lag + 1)]

    tmp_x = last_row[feature_cols].to_numpy()
    x_old = np.hstack([tmp_x[:2], y[-1], tmp_x[2:4]])
    
    year_list = [last_year + 1]
    country_list = [country]
    pred_list = [predict_next_year(beta_tourism, x_old)]

    for i in range(1, lag):
        year = last_year + i + 1
        
        X_new = np.hstack([x_old[:2], pred_list[i-1], x_old[2:4]])
        pred_tourism = predict_next_year(beta_tourism, X_new)

        year_list.append(year)
        country_list.append(country)
        pred_list.append(pred_tourism)

        x_old = X_new

    predictions = {'Year': year_list,
                   'Country': country_list,
                   'Predicted_Tourism': pred_list}

    return pd.DataFrame(predictions)


In [14]:
def reshape_predicted_data(data, st_data):

    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    print(min_trips, max_trips)

    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips

    rows_predicted = []
    for country in EU_countries_list:

        result_predicted = get_country_prediction(st_data, country, lag=3)
        result_predicted.sort_values(by=['Year'])
        arr_2024 = result_predicted[result_predicted["Year"] == 2024]["Predicted_Tourism"].values
        arr_2025 = result_predicted[result_predicted["Year"] == 2025]["Predicted_Tourism"].values
        arr_2026 = result_predicted[result_predicted["Year"] == 2026]["Predicted_Tourism"].values

        if arr_2024.size > 0:
            pred_2024 = float(arr_2024[0]) * (max_trips - min_trips) + min_trips
        else:
            pred_2024 = np.nan

        if arr_2025.size > 0:
            pred_2025 = float(arr_2025[0]) * (max_trips - min_trips) + min_trips
        else:
            pred_2025 = np.nan

        if arr_2026.size > 0:
            pred_2026 = float(arr_2026[0]) * (max_trips - min_trips) + min_trips
        else:
            pred_2026 = 0

        rows_predicted.append({'Countries': country, 
                     
                    2024: int(pred_2024), 
                    2025: int(pred_2025), 
                    2026: int(pred_2026)})
    

    country_name = st_data["Country"].iloc[0]
    data = data[data["Country"] == country_name]
        
    good_format_data_st = pd.DataFrame(rows_predicted)

    return good_format_data_st

In [ ]:
formatted_pred_data = reshape_predicted_data(data_not_st, data)
formatted_pred_data

,Countries,2024,2025,2026
0,Austria,34444302,-2236405,13441776
1,Belgium,62688551,92031534,122533776
2,Bulgaria,8851856,7245784,7040642
3,Croatia,11450306,11633468,9955771
4,Czechia,63799733,66528207,64343549
5,Denmark,42325525,44235158,45066051
6,Estonia,8972365,10627470,10464002
7,Finland,67748783,63476850,67441248
8,France,336346337,328110704,365125745
9,Germany,403537669,370498557,375861130


In [24]:
def reshape_historical_data(data_not_st):
    EU_countries_list = [
        'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
        "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
        "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
        "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
        "Spain", "Sweden"]
    
    eu_data = data_not_st[data_not_st['Country'].isin(EU_countries_list)]

    years = list(range(2012, 2024))   
    rows = []
    
    for country in EU_countries_list:
        row = {'Countries': country}
        for year in years:
            val = eu_data[
                (eu_data['Country'] == country) & (eu_data['Year'] == year)
            ]['NumTrips'].values
            row[year] = val[0] if val.size > 0 else np.nan
        rows.append(row)

    return pd.DataFrame(rows)


In [25]:
formatted_hist_data = reshape_historical_data(data_not_st)
formatted_hist_data

658396 526338516


,Countries,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,36003256.0,36479782.0,36681593,35815124,39365940,39201889,40524416,42415052,28583239,38854043,62381783,68320606.0
1,Belgium,20317974.0,24422291.0,24134015,25745332,29234314,27902578,30999148,33680649,22859689,30877766,46345022,49177986.0
2,Bulgaria,6847528.0,7485316.0,7157192,5828228,6839554,10535994,9777917,9477196,4999245,6279701,9092455,10297520.0
3,Croatia,14568947.0,15418760.0,13299645,11306996,7515349,8115498,8663271,10251772,6004684,7639926,9230247,11745012.0
4,Czechia,62812624.0,59216427.0,62884070,57463012,63214947,68826202,66349651,66969988,47497243,53928992,70746302,61610992.0
5,Denmark,52886592.0,58562414.0,60423616,60236946,54640232,47316730,30525585,52037652,42214364,32502703,37871415,37928868.0
6,Estonia,5027759.0,4899692.0,6571932,6587306,7486162,7661854,11040687,8753307,5153731,5437495,7625636,9425770.0
7,Finland,67795390.0,67102523.0,64541760,65963010,66412094,67995388,66187940,66165686,48632400,60007669,64711253,61106068.0
8,France,406369476.0,407204584.0,411807574,366221284,359227358,392347108,392229828,386709202,305202977,374563212,435660457,464286634.0
9,Germany,399531982.0,414453278.0,394377723,420166585,426990841,404592288,451416603,439169070,285977705,358428715,463342211,526338516.0


In [72]:
def visualization(formatted_hist_data, formatted_pred_data, country): 
    
    merged_wide = pd.merge(formatted_hist_data, formatted_pred_data, on='Countries', how='right')
    merged_wide = pd.DataFrame(merged_wide)

    year_cols = [col for col in merged_wide.columns if col != 'Countries']
    
    merged_long = merged_wide.melt(id_vars='Countries', value_vars=year_cols, var_name='Year', value_name='NumTrips')

    merged_long['Year'] = merged_long['Year'].astype(int)

    country_data = merged_long[merged_long['Countries'] == country].sort_values('Year')

    print(country_data)
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x= country_data['Year'], 
        y= country_data['NumTrips'], 
        name= f'Tourism Numbers in {country}',
        line= dict(color='royalblue', width=4),
    ))

    fig.add_vline(
        x=2023,  
        line=dict(color='gray', dash='dot'),
        annotation_text='Prediction',
        annotation_position='top right'
)
    fig.update_layout(
        title = f'Predicted Tourism Over the Years in {country}',
        title_x = 0.5, 
        xaxis_title = 'Years',
        yaxis_title = 'Tourism Numbers (Millions)')

    fig.show()

In [73]:
visualization(formatted_hist_data, formatted_pred_data, "Belgium")

    Countries  Year     NumTrips
1     Belgium  2012   20317974.0
27    Belgium  2013   24422291.0
53    Belgium  2014   24134015.0
79    Belgium  2015   25745332.0
105   Belgium  2016   29234314.0
131   Belgium  2017   27902578.0
157   Belgium  2018   30999148.0
183   Belgium  2019   33680649.0
209   Belgium  2020   22859689.0
235   Belgium  2021   30877766.0
261   Belgium  2022   46345022.0
287   Belgium  2023   49177986.0
313   Belgium  2024   62688551.0
339   Belgium  2025   92031534.0
365   Belgium  2026  122533776.0


In [70]:
pred_df_all = []
y_all = []
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]
#EU_countries_list = ['Slovakia']

# loop through all the countries
for country in EU_countries_list:
    country_df = data[data['Country'] == country].copy()
# for each country
# drop the last three years from data (call this subset_data or somethihg)
    max_year = country_df["Year"].max()
    if max_year < 2023:
        subset_data = country_df[country_df['Year'] < max_year - 1]
    else:
        subset_data = country_df[country_df['Year'] < max_year - 2]
    
    temp_df = get_country_prediction(subset_data, country, lag=3)
    #print(temp_df)

# concantenate temp_df to the bottom of pred_df
    pred_df_all.append(temp_df)

    #pred_df_all
# when you are done, compare the pred_df predictions to a vector y_true
# where y_true are the 2021-2023 y values for the countries
    if max_year < 2023:
        y_true = country_df[country_df['Year'] > max_year - 2]
    else:
        y_true = country_df[country_df['Year'] > max_year - 3]
    #print(y_true)
    y_all.append(y_true)

pred_df_true = pd.concat(pred_df_all, ignore_index=True)
y_fin = pd.concat(y_all, ignore_index=True)

results = pd.merge(pred_df_true, y_fin, on = ['Country', 'Year'])

results

results[results["Country"] == "Spain"]


,Year,Country,Predicted_Tourism,Tourism St.
72,2021,Spain,0.126263,0.423896
73,2022,Spain,-0.031529,0.523135
74,2023,Spain,-0.045168,0.552028


In [29]:
def evaluate_predictions(pred_df):
    r2_tourism = np.corrcoef(pred_df['Tourism St.'], pred_df['Predicted_Tourism'])[0,1]**2
    mse_tourism = mean_squared_error(pred_df['Tourism St.'], pred_df['Predicted_Tourism'])

    evaluations = {
        'Tourism': {'R2': r2_tourism, 'MSE': mse_tourism}
    }

    return evaluations

In [30]:
evaluate_predictions(results)

{'Tourism': {'R2': np.float64(0.06359236146683635), 'MSE': 0.2515251453378957}}